<a href="https://colab.research.google.com/github/mmurata112/ImageRecognition/blob/main/C964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Creating paths to necessary files in Google Drive
csv_file_path = "/content/drive/MyDrive/Colab Notebooks/sports2.csv"
requirements_path = "/content/drive/MyDrive/Colab Notebooks/requirements1.txt"
base_dir = "/content/drive/MyDrive/Colab Notebooks/sports_dataset"

# Loading sports data file and installing requirements.txt
sports_data = pd.read_csv(csv_file_path)
!pip install -r "{requirements_path}"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
import skimage
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import color
from IPython.display import clear_output

In [2]:
#Create dictionary that maps class id to sport
class_id_to_sport = {
    0: 'Football',
    1: 'Golf'
}

# Function that converts class id to sport
def convert_id_to_sport(ids):
    return [class_id_to_sport[id] for id in ids]

In [7]:
# Separate the datasets
train_dataset = sports_data[sports_data['data set'] == 'train']
valid_dataset = sports_data[sports_data['data set'] == 'valid']
test_dataset = sports_data[sports_data['data set'] == 'test']

In [ ]:
#Function to load and preprocess images
def load_and_preprocess_images(data, base_dir, img_size=(224, 224)):
    images = []
    labels = []
    hog_features = []
    # Iterate over rows of inputted data
    for index, row in data.iterrows():
        try:
            # Create full filepath to the images
            file_path = os.path.join(base_dir, row['filepaths'])
            # Read the image
            img = imread(file_path)
            # Resize the image
            img_resized = resize(img, img_size, anti_aliasing=True)
            # Normalize data
            img_normalized = img_resized / 255.0
            #Extract HOG features
            img_gray = color.rgb2gray(img_normalized)
            hog_feature = hog(img_gray, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)
            # Flatten the image
            img_flattened = img_normalized.flatten()
            # Append processed image, corresponding label, and HOG features
            labels.append(row['class id'])
            images.append(img_flattened)
            hog_features.append(hog_feature)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            continue

    #Combine pixel features with HOG features
    images_array = np.array(images)
    hog_features_array = np.array(hog_features)
    if images_array.shape[0] != hog_features_array.shape[0]:
        raise ValueError("Mismatch in number of images and HOG features.")
    else:
        features = np.concatenate((images_array, hog_features_array), axis=1)

    return features, np.array(labels)

In [ ]:
# Load and preprocess images and extract features and labels
X_train, y_train = load_and_preprocess_images(train_dataset, base_dir)
X_val, y_val = load_and_preprocess_images(valid_dataset, base_dir)
X_test, y_test = load_and_preprocess_images(test_dataset, base_dir)

In [ ]:
# Define parameter grid for RandomizedSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10, 100],
    'kernel': ['rbf']
}
# Initialize SVM and RandomizedSearchCV
svm = SVC(probability=True)
random_search = RandomizedSearchCV(svm, param_distributions=param_grid, n_iter=10, cv=3, n_jobs=-1, random_state=42)

# Fit model on the training data
print("Begin training...")
random_search.fit(X_train, y_train)
print("Training has ended")

# Use RandomizedSearchCV to find the best model
best_model = random_search.best_estimator_

Begin training...
Training has ended


In [ ]:
# Evaluate prediction and probability statistics on test/validation sets
y_test_pred = best_model.predict(X_test)
y_test_proba = best_model.predict_proba(X_test)

y_val_pred = best_model.predict(X_val)
y_val_proba = best_model.predict_proba(X_val)

# Generate classification_report function
def create_classification_report():
    print("Classification Report:")
    print(classification_report(y_test, y_test_pred, target_names=list(class_id_to_sport.values())))

In [ ]:
# Generate a confusion matrix function
def create_confusion_matrix():
    matrix = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
        xticklabels=list(class_id_to_sport.values()),
        yticklabels=list(class_id_to_sport.values()))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
# Compute ROC curve and ROC area
def create_roc_curve():
    # Compute ROC curve and AUC for test set
    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba[:, 1])
    roc_auc_test = auc(fpr_test, tpr_test)

    # Plot ROC curves
    plt.figure(figsize=(8, 6))
    plt.plot(fpr_test, tpr_test, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc_test)
    plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# Create function to display image with prediction
def display_image_with_prediction(index):
    # Extract the features from the test set
    features = X_test[[index]]

    # Calculate prediction and confidence score
    prediction = best_model.predict(features)[0]
    probability = best_model.predict_proba(features)[0, prediction]

    # Finding image and resizing
    img_path = os.path.join(base_dir, test_dataset.iloc[index]['filepaths'])
    img = imread(img_path)
    img_resized = resize(img, (224, 224), anti_aliasing=True)

    # Displaying image with prediction and confidence score
    plt.imshow(img_resized)
    plt.axis('off')
    plt.title(f"Prediction: {class_id_to_sport[prediction]} (Confidence: {probability:.2f})")
    plt.show()

    actual_label = y_test[index]
    is_correct = (prediction == actual_label)

    print("Correct Prediction!" if is_correct else "Incorrect Prediction.")
    print(f"Actual Label: {class_id_to_sport[actual_label]}")

In [ ]:
def menu():
    while True:
        clear_output(wait=True)
        print("Welcome to the Smith Photography Image Recoginition Application\nPlease choose from the following options:")
        print("1. Test/Display Image and Prediction")
        print("2. Display the Classification Report")
        print("3. Display the Confusion Matrix")
        print("4. Display the ROC Curve")
        print("5. Exit")
        choice = input("Please enter your choice: ")
        print()

        if choice == '1':
            try:
                index = int(input(f"Please enter image index number (0 to {len(test_dataset)-1}): "))
                if 0 <= index < len(test_dataset):
                    display_image_with_prediction(index)
                    choice = input("Would you like to choose another option? Type yes or no: ")
                    print()
                    if choice == "yes":
                        continue
                    else:
                        break
                else:
                    print("Index is out of range. Please enter valid index.")
            except ValueError:
                print("Invalid input")
        elif choice == '2':
            create_classification_report()
            choice = input("Would you like to choose another option? Type yes or no: ")
            print()
            if choice == "yes":
                continue
            else:
                break
        elif choice == '3':
            create_confusion_matrix()
            choice = input("Would you like to choose another option? Type yes or no: ")
            print()
            if choice == "yes":
                continue
            else:
                break
        elif choice == '4':
            create_roc_curve()
            choice = input("Would you like to choose another option? Type yes or no: ")
            print()
            if choice == "yes":
                continue
            else:
                break
        elif choice == '5':
            print("Thank you and goodbye!")
            break
        else:
            print("Invalid choice.  Please choose a number 1-5")
            print()
            continue

In [ ]:
menu()

Welcome to the Smith Photography Image Recoginition Application
Please choose from the following options:
1. Test/Display Image and Prediction
2. Display the Classification Report
3. Display the Confusion Matrix
4. Display the ROC Curve
5. Exit
Please enter your choice: 2

Classification Report:
              precision    recall  f1-score   support

    Football       0.80      0.80      0.80         5
        Golf       0.80      0.80      0.80         5

    accuracy                           0.80        10
   macro avg       0.80      0.80      0.80        10
weighted avg       0.80      0.80      0.80        10

Would you like to choose another option? Type yes or no: no

